<h1 align='center' style='color:black'> Week_6
Assignment</h1>

In [ ]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


### Write YAML file

In [ ]:
%%writefile file.yaml
file_type: csv
dataset_name: EpisodeAgentsfile
file_name: EpisodeAgents
#table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - Id
    - EpisodeId
    - Index
    - Reward
    - State
    - SubmissionId
    - InitialConfidence
    - InitialScore
    - UpdatedConfidence
    - UpdatedScore

Writing file.yaml


In [ ]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [ ]:
config_data['inbound_delimiter']

','

In [ ]:
#inspecting data of config file
config_data

{'columns': ['Id',
  'EpisodeId',
  'Index',
  'Reward',
  'State',
  'SubmissionId',
  'InitialConfidence',
  'InitialScore',
  'UpdatedConfidence',
  'UpdatedScore'],
 'dataset_name': 'EpisodeAgentsfile',
 'file_name': 'EpisodeAgents',
 'file_type': 'csv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1}

### Connecting to Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Normal reading process of the file
import pandas as pd
%time
df_sample = pd.read_csv('/content/drive/MyDrive/EpisodeAgents.csv',delimiter=',')
df_sample.head()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs


,Id,EpisodeId,Index,Reward,State,SubmissionId,InitialConfidence,InitialScore,UpdatedConfidence,UpdatedScore
0,89,45,0,1.0,2,13387740,NaN,NaN,NaN,NaN
1,90,45,1,0.0,2,13387739,NaN,NaN,NaN,NaN
2,119,52,0,0.0,2,13385797,NaN,NaN,NaN,NaN
3,123,52,1,1.0,2,13385792,NaN,NaN,NaN,NaN
4,113,59,0,1.0,2,13387710,NaN,NaN,NaN,NaN


In [ ]:
# read the file using config file
file_type = config_data['file_type']
source_file = "/content/drive/MyDrive/" + config_data['file_name'] + f'.{file_type}'
%time
df2 = pd.read_csv(source_file,config_data['inbound_d
elimiter'])
df2.head()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 9.3 µs


,Id,EpisodeId,Index,Reward,State,SubmissionId,InitialConfidence,InitialScore,UpdatedConfidence,UpdatedScore
0,89,45,0,1.0,2,13387740,NaN,NaN,NaN,NaN
1,90,45,1,0.0,2,13387739,NaN,NaN,NaN,NaN
2,119,52,0,0.0,2,13385797,NaN,NaN,NaN,NaN
3,123,52,1,1.0,2,13385792,NaN,NaN,NaN,NaN
4,113,59,0,1.0,2,13387710,NaN,NaN,NaN,NaN


In [ ]:
# Printing the Data Type for each Column and the capacity of the data on the memory
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44601868 entries, 0 to 44601867
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Id                 int64  
 1   EpisodeId          int64  
 2   Index              int64  
 3   Reward             float64
 4   State              int64  
 5   SubmissionId       int64  
 6   InitialConfidence  float64
 7   InitialScore       float64
 8   UpdatedConfidence  float64
 9   UpdatedScore       float64
dtypes: float64(5), int64(5)
memory usage: 3.3 GB


In [ ]:
df2.columns

Index(['Id', 'EpisodeId', 'Index', 'Reward', 'State', 'SubmissionId',
       'InitialConfidence', 'InitialScore', 'UpdatedConfidence',
       'UpdatedScore'],
      dtype='object')

In [ ]:
df2.shape

(44601868, 10)

In [ ]:
#validate the header of the file
util.col_header_val(df2,config_data)

column name and column length validation passed


1

In [ ]:
print("columns of files are:" ,df2.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['id', 'episodeid', 'index', 'reward', 'state', 'submissionid',
       'initialconfidence', 'initialscore', 'updatedconfidence',
       'updatedscore'],
      dtype='object')
columns of YAML are: ['Id', 'EpisodeId', 'Index', 'Reward', 'State', 'SubmissionId', 'InitialConfidence', 'InitialScore', 'UpdatedConfidence', 'UpdatedScore']


In [ ]:
if util.col_header_val(df2,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

column name and column length validation passed
col validation passed


In [ ]:
### Creating test file for this demo:
testdata = {
    'Id' : [89,90],
    'EpisodeId' : [45,45],
    'Index' : [1.0,0.0],
    'Reward' : [2,2],
    'State' : [13387740,13387739],

}
import pandas as pd
df = pd.DataFrame(testdata, columns=['Id', 'EpisodeId','Index','Reward','State'])
df.to_csv("/content/drive/MyDrive/data.csv",index=False)

In [ ]:
df

,Id,EpisodeId,Index,Reward,State
0,89,45,1.0,2,13387740
1,90,45,0.0,2,13387739


In [ ]:
testdata

{'EpisodeId': [45, 45],
 'Id': [89, 90],
 'Index': [1.0, 0.0],
 'Reward': [2, 2],
 'State': [13387740, 13387739]}

In [26]:
import gzip
import shutil
with open ('drive/MyDrive/Week_6.ipynb','rb') as f_input:
  with gzip.open ('week6.ipynb.gz', 'wb') as f_output:
    shutil.copyfileobj(f_input, f_output)
    print("Process Complete Successful")

Process Complete Successful
